# Recommender Systems Challenge, 2023/2024 @ PoliMi

# Introduction

## Problem Description
The application domain is book recommendation. The datasets contain interactions of users with books, in particular, if the user attributed to the book a rating of at least 4. The main goal of the competition is to discover which items (books) a user will interact with.

The datasets includes around 600k interactions, 13k users, 22k items (books).
The training-test split is done via random holdout, 80% training, 20% test.
The goal is to recommend a list of 10 potentially relevant items for each user.


## Datasets
All files are comma-separated (columns are separated with ',' ).

- **data_train.csv**:
Contains the training set, describing implicit preferences expressed by the users.
    - **user_id** : identifier of the user
    - **item_id** : identifier of the item (Book)
    - **data** : "1.0" if the user liked the book attributing it a rating of at least 4.


- **data_target_users_test.csv**:
Contains the ids of the users that should appear in your submission file.
The submission file should contain all and only these users.

- **sample_submission.csv**:
A sample submission file in the correct format: [user_id],[ordered list of recommended items].
Be careful with the spaces and be sure to recommend the correct number of items to every user.

# Requirements

The working environment and the required libraries are defined here.

In [127]:
# Import libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps # creation of sparse matrix
import matplotlib.pyplot as pyplot # plot

from tqdm import tqdm

from scipy.stats import uniform, loguniform
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import optuna as op

In [128]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


# Data Preprocessing

In [129]:
# Handle the input file
input_file = open('Input/data_train.csv', 'r')
type(input_file)

_io.TextIOWrapper

In [130]:
# Build the dataframe from the input file
input_df = pd.read_csv(
    filepath_or_buffer=input_file,
    dtype={0:int, 1:int, 2:int},
    names=['user_id','item_id','data'],
    header=0
)

input_df.shape

(478730, 3)

In [131]:
# Show the first rows of the dataframe
input_df.head(n=10)

,user_id,item_id,data
0,1,7,1
1,1,15,1
2,1,16,1
3,1,133,1
4,1,161,1
5,1,187,1
6,1,205,1
7,1,222,1
8,1,237,1
9,1,354,1


In [132]:
# Build the COO matrix with the COO constructor: (data, (row, column))
URM_all = sps.coo_matrix(
    (input_df["data"].values,
    (input_df["user_id"].values, input_df["item_id"].values))
)

URM_all

<13025x22348 sparse matrix of type '<class 'numpy.int32'>'
	with 478730 stored elements in COOrdinate format>

In [133]:
# Convert the matrix to Compressed Sparse Row format
URM_all = URM_all.tocsr()

URM_all

<13025x22348 sparse matrix of type '<class 'numpy.intc'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [134]:
# Set the number of folds to split
n_fold = 100

In [135]:
kf = KFold(n_splits=n_fold, shuffle=True, random_state=42)

In [136]:
for i, (_, v_ind) in enumerate(kf.split(input_df)):
    input_df.loc[input_df.index[v_ind], 'kfold'] = int(i)

In [137]:
input_df.head(10)

,user_id,item_id,data,kfold
0,1,7,1,36.0
1,1,15,1,62.0
2,1,16,1,7.0
3,1,133,1,73.0
4,1,161,1,51.0
5,1,187,1,96.0
6,1,205,1,1.0
7,1,222,1,26.0
8,1,237,1,65.0
9,1,354,1,63.0


In [138]:
df_trains = []
df_tests = []

for i in range(n_fold):
    df_trains.append(input_df[input_df["kfold"]!=i])
    df_tests.append(input_df[input_df["kfold"]==i])
    
    df_trains[i] = df_trains[i].drop(columns=['kfold'])
    df_tests[i] = df_tests[i].drop(columns=['kfold'])

In [139]:
df_trains[0]

,user_id,item_id,data
0,1,7,1
1,1,15,1
2,1,16,1
3,1,133,1
4,1,161,1
...,...,...,...
478725,13024,13605,1
478726,13024,13823,1
478727,13024,15122,1
478728,13024,18185,1


In [140]:
URM_trains = []
URM_tests = []


for i in range(n_fold):
    URM_trains.append(sps.coo_matrix(
    (df_trains[i]["data"].values,
    (df_trains[i]["user_id"].values, df_trains[i]["item_id"].values))
).tocsr())
    
    
    URM_tests.append(sps.coo_matrix(
    (df_tests[i]["data"].values,
    (df_tests[i]["user_id"].values, df_tests[i]["item_id"].values))
).tocsr())

In [141]:
# # Save the URMs
# for i in range(n_fold):
#     sps.save_npz(f'{"kftrain" + str(i)}', URM_trains[i], compressed=True)
#     sps.save_npz(f'{"kftest" + str(i)}', URM_tests[i], compressed=True)

In [142]:
URM_trains[0]

<13025x22348 sparse matrix of type '<class 'numpy.intc'>'
	with 473942 stored elements in Compressed Sparse Row format>

In [143]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluators = []

for i in range(n_fold):
    evaluators.append(EvaluatorHoldout(URM_tests[i], cutoff_list=[10]))

EvaluatorHoldout: Ignoring 9852 (75.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9921 (76.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9895 (76.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9926 (76.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9910 (76.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9967 (76.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9919 (76.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9831 (75.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9881 (75.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9895 (76.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9921 (76.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9927 (76.2%) Users that hav

In [144]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

In [145]:
slimes = []

for i in range(n_fold):
    slime = MultiThreadSLIM_SLIMElasticNetRecommender(URM_trains[i])
    slime.fit(topK=692, alpha=0.000722, l1_ratio=0.0702)
    slimes.append(slime)

SLIMElasticNetRecommender: URM Detected 396 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 130 ( 0.6%) items with no interactions.


100%|██████████████████████████████████████████████████████████████████████████▉| 22344/22348 [02:23<00:00, 155.37it/s]


SLIMElasticNetRecommender: URM Detected 394 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 130 ( 0.6%) items with no interactions.


100%|██████████████████████████████████████████████████████████████████████████▉| 22344/22348 [03:09<00:00, 117.67it/s]


SLIMElasticNetRecommender: URM Detected 395 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 129 ( 0.6%) items with no interactions.


...

100%|██████████████████████████████████████████████████████████████████████████▉| 22344/22348 [03:11<00:00, 116.84it/s]


SLIMElasticNetRecommender: URM Detected 393 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 127 ( 0.6%) items with no interactions.


100%|██████████████████████████████████████████████████████████████████████████▉| 22344/22348 [03:09<00:00, 117.69it/s]


SLIMElasticNetRecommender: URM Detected 396 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 133 ( 0.6%) items with no interactions.


100%|██████████████████████████████████████████████████████████████████████████▉| 22344/22348 [02:30<00:00, 148.61it/s]


In [146]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.Similarity.Compute_Similarity import Compute_Similarity

In [147]:
itemhybrids = []

for i in range(n_fold):
    sim1 = Compute_Similarity(URM_trains[i], shrink = 8.516265, topK = 112, similarity = "cosine").compute_similarity()
    sim2 = Compute_Similarity(URM_trains[i], shrink = 4.174645, topK = 3, similarity = "dice").compute_similarity()

    itemhybrid = ItemKNNSimilarityHybridRecommender(URM_trains[i], sim1, sim2)
    itemhybrid.fit(topK = 68, alpha = 0.212)
    itemhybrids.append(itemhybrid)

Similarity column 22348 (100.0%), 10087.88 column/sec. Elapsed time 2.22 sec
Similarity column 22348 (100.0%), 9757.67 column/sec. Elapsed time 2.29 sec
ItemKNNSimilarityHybridRecommender: URM Detected 396 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10093.92 column/sec. Elapsed time 2.21 sec
Similarity column 22348 (100.0%), 9818.66 column/sec. Elapsed time 2.28 sec
ItemKNNSimilarityHybridRecommender: URM Detected 394 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10144.91 column/sec. Elapsed time 2.20 sec
Similarity column 22348 (100.0%), 9839.48 column/sec. Elapsed time 2.27 sec
ItemKNNSimilarityHybridRecommender: URM Detected 395 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 129 ( 0.6%) items with no interactions.
Similar

Similarity column 22348 (100.0%), 10218.26 column/sec. Elapsed time 2.19 sec
Similarity column 22348 (100.0%), 9928.35 column/sec. Elapsed time 2.25 sec
ItemKNNSimilarityHybridRecommender: URM Detected 396 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 132 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10173.55 column/sec. Elapsed time 2.20 sec
Similarity column 22348 (100.0%), 9965.04 column/sec. Elapsed time 2.24 sec
ItemKNNSimilarityHybridRecommender: URM Detected 399 ( 3.1%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 128 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10218.52 column/sec. Elapsed time 2.19 sec
Similarity column 22348 (100.0%), 10020.95 column/sec. Elapsed time 2.23 sec
ItemKNNSimilarityHybridRecommender: URM Detected 397 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 131 ( 0.6%) items with no interactions.
Simila

Similarity column 22348 (100.0%), 10125.63 column/sec. Elapsed time 2.21 sec
Similarity column 22348 (100.0%), 9975.44 column/sec. Elapsed time 2.24 sec
ItemKNNSimilarityHybridRecommender: URM Detected 394 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10159.77 column/sec. Elapsed time 2.20 sec
Similarity column 22348 (100.0%), 9925.57 column/sec. Elapsed time 2.25 sec
ItemKNNSimilarityHybridRecommender: URM Detected 398 ( 3.1%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 129 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10130.55 column/sec. Elapsed time 2.21 sec
Similarity column 22348 (100.0%), 9868.22 column/sec. Elapsed time 2.26 sec
ItemKNNSimilarityHybridRecommender: URM Detected 395 ( 3.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 128 ( 0.6%) items with no interactions.
Similar

Similarity column 22348 (100.0%), 10147.54 column/sec. Elapsed time 2.20 sec
Similarity column 22348 (100.0%), 9860.91 column/sec. Elapsed time 2.27 sec
ItemKNNSimilarityHybridRecommender: URM Detected 399 ( 3.1%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 127 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10112.26 column/sec. Elapsed time 2.21 sec
Similarity column 22348 (100.0%), 9928.60 column/sec. Elapsed time 2.25 sec
ItemKNNSimilarityHybridRecommender: URM Detected 401 ( 3.1%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 132 ( 0.6%) items with no interactions.
Similarity column 22348 (100.0%), 10164.94 column/sec. Elapsed time 2.20 sec
Similarity column 22348 (100.0%), 9927.11 column/sec. Elapsed time 2.25 sec
ItemKNNSimilarityHybridRecommender: URM Detected 400 ( 3.1%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
Similar

In [148]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

In [149]:
rp3betas = []

for i in range(n_fold):
    rp3beta = RP3betaRecommender(URM_trains[i])
    rp3beta.fit(topK=25, alpha=0.253519, beta=0.209538, implicit=True,
               normalize_similarity=True)
    rp3betas.append(rp3beta)

RP3betaRecommender: URM Detected 396 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 130 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5648.23 column/sec. Elapsed time 3.96 sec
RP3betaRecommender: URM Detected 394 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 130 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5632.29 column/sec. Elapsed time 3.97 sec
RP3betaRecommender: URM Detected 395 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 129 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5567.95 column/sec. Elapsed time 4.01 sec
RP3betaRecommender: URM Detected 398 ( 3.1%) users with no interactions.
RP3betaRecommender: URM Detected 130 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5622.33 column/sec. Elapsed time 3.97 sec
RP3betaRecommender: URM Detected

RP3betaRecommender: URM Detected 397 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 129 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5668.17 column/sec. Elapsed time 3.94 sec
RP3betaRecommender: URM Detected 400 ( 3.1%) users with no interactions.
RP3betaRecommender: URM Detected 129 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5670.37 column/sec. Elapsed time 3.94 sec
RP3betaRecommender: URM Detected 397 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 128 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5649.68 column/sec. Elapsed time 3.96 sec
RP3betaRecommender: URM Detected 401 ( 3.1%) users with no interactions.
RP3betaRecommender: URM Detected 128 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5679.50 column/sec. Elapsed time 3.93 sec
RP3betaRecommender: URM Detected

RP3betaRecommender: URM Detected 396 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 130 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5670.58 column/sec. Elapsed time 3.94 sec
RP3betaRecommender: URM Detected 393 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 127 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5621.18 column/sec. Elapsed time 3.98 sec
RP3betaRecommender: URM Detected 390 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 133 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5661.78 column/sec. Elapsed time 3.95 sec
RP3betaRecommender: URM Detected 402 ( 3.1%) users with no interactions.
RP3betaRecommender: URM Detected 129 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22348 (100.0%), 5650.31 column/sec. Elapsed time 3.96 sec
RP3betaRecommender: URM Detected

In [150]:
from Recommenders.GeneralScoresHybridRecommender import GeneralScoresHybridRecommender

In [151]:
generals = []

for i in range(n_fold):
    general = GeneralScoresHybridRecommender(
        URM_trains[i], 
        [slimes[i], rp3betas[i], itemhybrids[i]])
    general.fit([0.7, 0.25, 0.05])
    
    generals.append(general)

GeneralScoresHybridRecommender: URM Detected 396 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 394 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 395 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 129 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 398 ( 3.1%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 399 ( 3.1%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 134 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 403 ( 3.1%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 129 ( 0.6%) items wi

GeneralScoresHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 393 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 131 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 396 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 133 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 396 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 132 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 400 ( 3.1%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 130 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 395 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 133 ( 0.6%) items with no interactions.
GeneralScoresHybridRecommender: URM Detected 398 ( 3.1%) users wi

In [124]:
final = GeneralScoresHybridRecommender(
    URM_all, generals
)

final.fit([1/n_fold] * n_fold)

GeneralScoresHybridRecommender: URM Detected 387 ( 3.0%) users with no interactions.
GeneralScoresHybridRecommender: URM Detected 126 ( 0.6%) items with no interactions.


In [125]:
recommender = final

In [86]:
def recommendAll(recommender_array, id, cutoff=10):
    
    recommendations = []
    times = []
    result = []
    
    for rec in recommender_array:
        rec_list = rec.recommend(id, cutoff=10)
        
        for elem in rec_list:
            if elem in recommendations:
                times[recommendations.index(elem)] += 1
            else:
                recommendations.append(elem)
                times.append(1)

    for i in range(max(times), 0, -1):
        if(len(result)<10):
            for n in range(len(times)):
                if times[n] == i:
                    result.append(recommendations[n])
                    if(len(result) == 10):
                        break
        else:
            break
            
        
    return result

In [50]:
res = recommendAll(generals, 2, cutoff=10)

In [51]:
res

[1095, 47, 1522, 50, 12, 359, 54, 656, 3176, 196]

# Matrix Conversion

# Evaluation of the recommender

In [ ]:
# Define the train-test splitting ratio
train_test_split = 0.85

In [ ]:
# Import Evaluators
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [ ]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = train_test_split)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = train_test_split)


# evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

Content-based: URM in csr, ICM in csr

Collaborative filtering: URM in csr

FunkSVD: URM in coo

AsySVD: URM in coo


In [ ]:
result, _ = evaluator_test.evaluateRecommender(recommender)
result.loc[10]

# Recommend items

In [152]:
# Open the submission sample file
submission_sample = open('Input/data_target_users_test.csv', 'r')
type(submission_sample)

_io.TextIOWrapper

In [153]:
# Read the dataframe
submission_df = pd.read_csv(filepath_or_buffer=submission_sample)
submission_df

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
10877,13020
10878,13021
10879,13022
10880,13023


In [154]:
# Convert the user_id column to numpy list
user_list = submission_df['user_id'].to_numpy()
user_list

array([    1,     2,     3, ..., 13022, 13023, 13024], dtype=int64)

In [126]:
# # Extract the recommended items and write them in the dataframe

# all_recommend = []

# for i,id in tqdm(enumerate(user_list)):
#     rec = recommender.recommend(id, cutoff=10)
#     rec_list = rec
#     rec_row = ' '.join(str(s) for s in rec_list)
#     all_recommend.append(rec_row)

0it [00:00, ?it/s]


ValueError: operands could not be broadcast together with shapes (1,22348) (1,22347) (1,22348) 

In [155]:
# Extract the recommended items and write them in the dataframe

all_recommend = []

for i,id in tqdm(enumerate(user_list)):
    rec = recommendAll(generals, id, cutoff=10)
    rec_list = rec
    rec_row = ' '.join(str(s) for s in rec_list)
    all_recommend.append(rec_row)

10882it [15:09, 11.96it/s]


In [156]:
all_recommend[:10]

['36 101 403 506 515 123 694 52 1546 1422',
 '1095 47 12 50 1522 949 102 196 3176 54',
 '59 2172 4252 857 4 648 1281 259 4623 956',
 '249 28 50 171 314 146 7 136 254 128',
 '1570 170 1511 471 77 95 5138 131 1220 8',
 '886 874 35 395 88 168 9 104 9329 184',
 '210 443 451 95 600 3916 1094 722 480 1130',
 '9018 2821 2282 17799 10108 2350 4535 115 20151 21322',
 '1816 1446 561 1668 2617 2565 3905 2423 67 1767',
 '31 40 58 67 34 955 185 52 32 514']

In [157]:
submission_df['item_list'] = all_recommend

In [158]:
submission_df.head(10)

,user_id,item_list
0,1,36 101 403 506 515 123 694 52 1546 1422
1,2,1095 47 12 50 1522 949 102 196 3176 54
2,3,59 2172 4252 857 4 648 1281 259 4623 956
3,4,249 28 50 171 314 146 7 136 254 128
4,5,1570 170 1511 471 77 95 5138 131 1220 8
5,6,886 874 35 395 88 168 9 104 9329 184
6,8,210 443 451 95 600 3916 1094 722 480 1130
7,9,9018 2821 2282 17799 10108 2350 4535 115 20151...
8,10,1816 1446 561 1668 2617 2565 3905 2423 67 1767
9,11,31 40 58 67 34 955 185 52 32 514


In [159]:
# Export the dataframe on csv file
submission_df.to_csv('Output/KFold100.csv', index=False)